# [OpenCV-Python Tutorial] Homography

In this notebook, we will learn geometric transformation of images in OpenCV-Python and how to get a homography of two images using SIFT feature matching.

---

In [ ]:
import numpy as np
import cv2 # OpenCV-Python
%matplotlib inline
import matplotlib.pyplot as plt
from skimage import io

plt.style.use('default')
print("OpenCV-Python Version %s" % cv2.__version__)
home_url = r'https://raw.githubusercontent.com/InovaDx/public/master/OpenCV/'

### Loading images
Loading images from a local disk unit is done with the following code:
```python
input_image = cv2.imread('filepath/image.png', cv2.IMREAD_COLOR)
```
When running this notebook from Google Colaboratory, images are not local, and the way to get to open them changes. A different library must be used (`skimage`) in order to fetch the hyperlink to the image and open it as an array of pixels.
One of the main differences between both methods is the fact that `OpenCV`, unless specified, always opens images as RGB arrays with 3 layers, while `skimage` only creates the required layers.
That results in some inconsistencies in case a grayscale image is loaded, since only 1 layer is loaded by `skimage`, but `OpenCV` loads all 3 RGB.

This notebook assumes it is used from Google Colab and will always use the `skimage` option, even though is less common than using `OpenCV` in real life.

# Geometric Transformations of Images

In [ ]:
# Scaling
filename='images/messi.jpg'
img = io.imread(home_url+filename)
height, width = img.shape[:2]
res = cv2.resize(img,(2*width, 2*height), interpolation = cv2.INTER_CUBIC)

plt.figure(figsize=(10,3))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.imshow(res)

**NOTE**
The code to load an image from a local disk unit in grayscale mode is:
```python
img = cv2.imread('images/messi.jpg', cv2.IMREAD_GRAYSCALE)
```
However, the use of images in the Internet requires an alternative way to convert the image to grayscale:
```python
cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
```

In [ ]:
# Translation
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
rows,cols = img.shape

M = np.float32([[1,0,100],[0,1,50]])
dst = cv2.warpAffine(img,M,(cols,rows))

plt.figure(figsize=(10,3))
plt.subplot(1, 2, 1)
plt.imshow(img, cmap='gray')
plt.subplot(1, 2, 2)
plt.imshow(dst, cmap='gray')

In [ ]:
# Rotation
rows,cols = img.shape

M = cv2.getRotationMatrix2D((cols/2,rows/2),90,1)
print (M)
dst1 = cv2.warpAffine(img,M,(cols,rows))
dst2 = cv2.warpAffine(img,M,(cols,rows),borderMode=cv2.BORDER_WRAP)
dst3 = cv2.warpAffine(img,M,(cols,rows),borderMode=cv2.BORDER_REFLECT)

plt.figure(figsize=(16,3))
plt.subplot(1, 4, 1)
plt.imshow(img, cmap='gray')
plt.subplot(1, 4, 2)
plt.imshow(dst1, cmap='gray')
plt.subplot(1, 4, 3)
plt.imshow(dst2, cmap='gray')
plt.subplot(1, 4, 4)
plt.imshow(dst3, cmap='gray')

In [ ]:
# Affine Transformation
img = io.imread(home_url+'images/drawing.png')
rows,cols,ch = img.shape

pts1 = np.float32([[50,50],[200,50],[50,200]])
pts2 = np.float32([[10,100],[200,50],[100,250]])

M = cv2.getAffineTransform(pts1,pts2)
print (M)

dst = cv2.warpAffine(img,M,(cols,rows))

plt.subplot(1, 2, 1)
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.imshow(dst)

In [ ]:
cv2.warpPerspective?

In [ ]:
# Homography(Perspective Transformation)
img1 = io.imread(home_url+'images/track.jpg')
img2 = io.imread(home_url+'images/logo.jpg')
rows1,cols1,ch1 = img1.shape
rows2,cols2,ch2 = img2.shape

pts1 = np.float32([(0,0),(cols2-1,0),(cols2-1,rows2-1),(0,rows2-1)])
pts2 = np.float32([(671,314),(1084,546),(689,663),(386,361)])

M = cv2.getPerspectiveTransform(pts1,pts2)
print (M)
img3 = np.copy(img1)
cv2.warpPerspective(img2,M,(cols1,rows1),img3,borderMode=cv2.BORDER_TRANSPARENT)

plt.figure(figsize=(14,3))
plt.subplot(1, 3, 1)
plt.imshow(img1)
plt.subplot(1, 3, 2)
plt.imshow(img2)
plt.subplot(1, 3, 3)
plt.imshow(img3)

# Homography from SIFT Matching

## 1. SIFT matching from the previous notebook

In [ ]:
# Open and show images
img1 = io.imread(home_url+'images/box.png')
img2 = io.imread(home_url+'images/box_in_scene.png')

# SIFT feature extracting
sift = cv2.xfeatures2d.SIFT_create()
gray1 = img1
gray2 = img2
if len(img1.shape)==3:
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
if len(img2.shape)==3:
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

# BFMatcher(Brute Force Matcher) with defalut setting
bf = cv2.BFMatcher(cv2.NORM_L2)
matches = bf.knnMatch(des1, des2, k=2)

# Apply ratio test as in David Rowe's paper
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)
print('%d matches' % len(good_matches))

img3 = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None)
plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))

## 2. Get Homography Matrix & Object Detection

In [ ]:
# Matched points
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

print (src_pts[:3])
print (dst_pts[:3])

In [ ]:
# Find homography matrix with RANSAC algorithm
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
matchesMask = mask.ravel().tolist()
print (M)

In [ ]:
# Calculate the object position in the scene using homography
h,w = img1.shape[0:2]
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts,M)
print (dst)

In [ ]:
# Display the object
img2 = cv2.polylines(img2,[np.int32(dst)],True,(255,0,0),3, cv2.LINE_AA)
plt.imshow(img2, cmap='gray')

In [ ]:
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

img3 = cv2.drawMatches(img1,kp1,img2,kp2,good_matches,None,**draw_params)
plt.imshow(img3)

In [ ]:
# Display the object
img1 = cv2.polylines(img1,[np.int32(pts)],True,(255,0,0),3, cv2.LINE_AA)
img2 = cv2.polylines(img2,[np.int32(dst)],True,(255,0,0),3, cv2.LINE_AA)

draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   flags = 2)

img3 = cv2.drawMatches(img1,kp1,img2,kp2,good_matches,None,**draw_params)
plt.imshow(img3)